## Auto Gen Tutorial
Note book written by John Adeojo
Founder, and Chief Data Scientist at [Data-centric Solutions](https://www.data-centric-solutions.com/)


In [39]:
import os
import yaml
import openai

script_dir = "C:/Users/johna/OneDrive/Documents/api_keys/"

# Mount Google Drive
# drive.mount('/content/drive')

def get_apikey(script_dir=script_dir):
    """
    Reads API key from a configuration file.

    This function opens a configuration file named "apikeys.yml", reads the API key for OpenAI

    Returns:
    api_key (str): The OpenAI API key.
    """
    # Update the script_dir path to point to the correct location in your Google Drive
    script_dir = script_dir
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as yamlfile:
        loaded_yamlfile = yaml.safe_load(yamlfile)
        API_KEY = loaded_yamlfile['openai']['api_key']

    return API_KEY

# Call the function to get the API key
openai.api_key = get_apikey()

In [40]:
import autogen 

index_path = "G:/My Drive/Data-Centric Solutions/07. Blog Posts/AutoGen/autogen_tutorial/indexes/"
configurations_path = "G:/My Drive/Data-Centric Solutions/07. Blog Posts/AutoGen/autogen_tutorial/"

config_list = autogen.config_list_from_json(
    env_or_file="configurations.json",
    file_location=configurations_path,
    filter_dict={
        "model": ["gpt-3.5-tubro-16k"],
    },
)

In [41]:
# Tool 1: Does a query based search for Wikipages
from typing import Any, List
import wikipedia
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.text_splitter import get_default_text_splitter
import openai
from pydantic import BaseModel
from llama_index.program import OpenAIPydanticProgram
from utils import get_apikey
from typing import Callable, Dict, Optional, Union, List, Tuple, Any
from llama_hub.wikipedia.base import WikipediaReader
from llama_index import StorageContext
from llama_index import load_index_from_storage
import json
from autogen import oai

class WikiPageList(BaseModel):
    "Data model for WikiPageList"
    pages: list

def wikipage_list(wikipediapages):
        openai.api_key = get_apikey()

        prompt_template_str = """
        Given the input {query}, 
        return the list of Wikipedia pages.
        """
        program = OpenAIPydanticProgram.from_defaults(
            output_cls=WikiPageList,
            prompt_template_str=prompt_template_str,
            verbose=True
        )
        return program(query=wikipediapages)

def search_wikipedia(
        query: str, lang: str = "en", results_limit: int = 5, **load_kwargs: Any
    ) -> List[List]:
    wikipedia.set_lang(lang)
    return wikipedia.search(query, results=results_limit)

def create_wikidocs(query: str) -> list:  # Added type hint
    loader = WikipediaReader()  # Removed redefinition of WikipediaReader
    return loader.load_data(pages=query)

def save_index(index) -> None:
    # index.set_index_id("vector_index")
    index.storage_context.persist(index_path)

def index_wikipedia_pages(wikipediapages: str) -> VectorStoreIndex:  # Added type hint
    global index
    wikipageslist = wikipage_list(wikipediapages)
    documents = create_wikidocs(wikipageslist)
    text_splits = get_default_text_splitter(chunk_size=150, chunk_overlap=45)
    parser = SimpleNodeParser.from_defaults(text_splitter=text_splits)
    service_context = ServiceContext.from_defaults(node_parser=parser)
    index =  VectorStoreIndex.from_documents(documents, service_context=service_context)
    save_index(index)


def load_index(filepath: str):
        # rebuild storage context
        # storage_context = StorageContext.from_defaults(persist_dir="indexes")
        storage_context = StorageContext.from_defaults(persist_dir=index_path)
        # load index
        return load_index_from_storage(storage_context)

def read_json_file(file_path: str) -> dict:
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def query_vector_db(
        search_string: str,
        file_path = index_path,
        n_results: int = 10
        # **kwargs,
) -> None:  # Updated return type to None as the function now writes to a file
    index = load_index(filepath=index_path)
    query_engine = index.as_query_engine(
        response_mode="compact", verbose=True, similarity_top_k=n_results
    )
    nodes = query_engine.query(search_string).source_nodes
    retrieved_context = {
        # "ids": [],
        "text": []
    }
    for node in nodes:
        # retrieved_context["ids"].append(node.node.id_)
        retrieved_context["text"].append(node.node.text)
    
    file_path = index_path + "retrieved_context.json"
    with open(file_path, 'w') as f:
        json.dump(retrieved_context, f)
    
    return retrieved_context

# def generate_response(task:str, config_list=config_list):
#      context = read_json_file(index_path + "retrieved_context.json")
#      response = oai.Completion.create(
#           config_list=config_list,
#           prompt=f"Use `{context}` to help you respond to the `{task}`"
#      )
#      return response

def generate_response(task:str):
    openai.api_key = get_apikey()
    context = read_json_file(index_path + "retrieved_context.json")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",  
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": task},
            {"role": "assistant", "content": f"Use the information `{context}` to help you respond to the `{task}`"}
        ]
    )
    return response


# # Call the function
# def query_vector_db(
#         n_results: int = 20,
#         search_string: str = "",
#         # **kwargs,
# ) -> Dict[str, Union[List[str], List[List[str]]]]:
#     openai.api_key = get_apikey()
#     index = load_index(filepath=index_path)
#     query_engine = index.as_query_engine(
#         response_mode="compact", verbose=True, similarity_top_k=n_results
#     )
#     nodes = query_engine.query(search_string).source_nodes
#     results = {
#         "ids": [],
#         "text": []
#     }
#     for node in nodes:
#         results["ids"].append(node.node.id_)
#         results["text"].append(node.node.text)
#     return results

# def retrieve_docs(problem: str, n_results: int = 20, search_string: str = "", **kwargs):
#     results = query_vector_db(problem, n_results, search_string)
#     return results
#     print("doc_ids: ", results["ids"])

In [42]:
llm_config = {
    "functions": [
        {
            "name": "search_wikipedia",
            "description": "Use this to search for relevant Wikipedia pages",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "A query to search and identify relevant Wikipedia pages",
                    }
                },
                "required": ["query"],
            },
        },
        
        {
            "name": "index_wikipedia_pages",
            "description": "Use this to index wikipedia pages",
            "parameters": {
                "type": "object",
                "properties": {
                    "wikipediapages": {
                        "type": "string",
                        "description": "The output of the search_wikipedia function",
                    }
                },
                "required": ["wikipediapages"],
            },
        },

        {
            "name": "query_vector_db",
            "description": "Useful for retrieving relevant context from the index",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_string": {
                        "type": "string",
                        "description": "A query for which relevant context is required from the index",
                    }
                },
                "required": ["search_string"],
            },
        },

        {
            "name": "generate_response",
            "description": "Useful to generate a response based on additional context provided by wikipedia search",
            "parameters": {
                "type": "object",
                "properties": {
                    "task": {
                        "type": "string",
                        "task": "Your understanding of the task",
                    }
                },
                "required": ["task"],
            },
        },
        
    ],
    "config_list": config_list,
    "request_timeout": 120,
}

In [43]:
# from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
# import openai
# from utils import get_apikey
# from llama_index import StorageContext
# from llama_index import load_index_from_storage


# class WikipediaRetrieverUserProxyAgent(RetrieveUserProxyAgent):
#     def __init__(self, index_path=index_path, **kwargs):
#         super().__init__(**kwargs)  # Fixed superclass constructor call
#         # openai.api_key = get_apikey()
#         self.index_path = index_path

#     @staticmethod
#     def load_index(filepath: str):
#         # rebuild storage context
#         storage_context = StorageContext.from_defaults(persist_dir="indexes")
#         # load index
#         return load_index_from_storage(storage_context, index_id="vector_index")
    
#     def query_vector_db(
#             self,
#             n_results: int = 20,
#             search_string: str = "",
#             **kwargs,
#     ) -> Dict[str, Union[List[str], List[List[str]]]]:
#         index = self.load_index(self.index_path)
#         query_engine = index.as_query_engine(
#             response_mode="compact", verbose=True, similarity_top_k=n_results
#         )
#         nodes = query_engine.query(search_string).source_nodes
#         results = {
#             "ids": [],
#             "text": []
#         }
#         for node in nodes:
#             results["ids"].append(node.node.id_)
#             results["text"].append(node.node.text)
#         return results 

#     def retrieve_docs(self, problem: str, n_results: int = 20, search_string: str = "", **kwargs):
#         results = self.query_vector_db(problem, n_results, search_string)
#         self._results = results
#         print("doc_ids: ", results["ids"])

# # class WikipediaRetrieverUserProxyAgent(RetrieveUserProxyAgent):
# #     def __init__(self, index, **kwargs):
# #         super().__init__(**kwargs)  # Fixed superclass constructor call
# #         # Set the OpenAI API key when the class is instantiated
# #         openai.api_key = get_apikey()
# #         self.index=index

# #     def query_vector_db(
# #             self,
# #             n_results: int = 20,
# #             search_string: str = "",
# #             **kwargs,
# #     ) -> Dict[str, Union[List[str], List[List[str]]]]:
# #         query_engine = self.index.as_query_engine(
# #             response_mode="compact", verbose=True, similarity_top_k=n_results
# #         )
# #         nodes = query_engine.query(search_string)
# #         results = {
# #             "ids": [],
# #             "text": []
# #         }
# #         for node in nodes:
# #             results["ids"].append(node.node.id_)
# #             results["text"].append(node.node.text) 
    
# #     def retrieve_docs(self, problem: str, n_results: int = 20, search_string: str = "", **kwargs):
# #         results = self.query_vector_db(problem, n_results, search_string)
# #         self._results = results
# #         print("doc_ids: ", results["ids"])


In [44]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    system_message= "You are the manager that ensures all the agents take neccessary steps to respond to the task.",
    code_execution_config={"work_dir": "coding"},
)

wikisearch_agent = autogen.AssistantAgent(
    name="wikisearch_agent",
    system_message="Your job is to search for Wikipedia pages relevant to the task with the search_wikipedia tool.",
    llm_config=llm_config,
)

# wikise_agent = autogen.AssistantAgent(
#     name="wikisearch_agent",
#     system_message="Your job is to search for relevant Wikipedia pages with the search_wikipedia tool.",
#     llm_config=llm_config,
# )

wiki_index_agent = autogen.AssistantAgent(
    name="wiki_index_agent",
    system_message="Index relevant Wikipedia pages using the index_wikipedia_pages tool as suggested by the wikisearch_agent",
    llm_config=llm_config,
)

retrieve_agent = autogen.AssistantAgent(
    name="retrieve_agent",
    system_message="Retrieve relevant from the index information with the query_vector_db tool and communicate it",
    llm_config=llm_config,
)

response_agent = autogen.AssistantAgent(
    name="response_agent",
    system_message="Generate a response to the task based using the generate_response tool.",
    llm_config=llm_config,
)


# retrieve_response_agent = WikipediaRetrieverUserProxyAgent(
#     name="retrieve_response_agent",
#     system_message="Generate response based on context",
#     human_input_mode="NEVER",
#     llm_config=llm_config,
#     index_path=index_path
    
# )

user_proxy.register_function(
    function_map={
        "search_wikipedia": search_wikipedia,
        "index_wikipedia_pages":index_wikipedia_pages,
        "query_vector_db":query_vector_db,
        "generate_response":generate_response
    }
)

groupchat = autogen.GroupChat(agents=[user_proxy, wikisearch_agent, wiki_index_agent, retrieve_agent, response_agent], messages=[], max_round=100)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(manager, message="What is the population of London?")


user_proxy (to chat_manager):

What is the population of London?

--------------------------------------------------------------------------------
[autogen.oai.completion: 11-03 18:53:30] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
[autogen.oai.completion: 11-03 18:53:31] {788} WARNING - Completion was provided with a config_list, but the list was empty. Adopting default OpenAI behavior, which reads from the 'model' parameter instead.
wikisearch_agent (to chat_manager):

***** Suggested function Call: search_wikipedia *****
Arguments: 
{
  "query": "Population of London"
}
*****************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_wikipedia...
user_proxy (to chat_manager):

***** Response from calling function "search_wikipedia" *****
['Demography of Lond